In [0]:
import numpy as np
import pandas as pd

In [4]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# tfidf = dict(list(zip('cat dog apple lion NYC love'.split(), np.random.random(5))))

In [2]:
!git clone https://github.com/totalgood/nlpia.git

Cloning into 'nlpia'...
remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 5804 (delta 84), reused 118 (delta 49), pack-reused 5631
Receiving objects: 100% (5804/5804), 124.00 MiB | 28.81 MiB/s, done.
Resolving deltas: 100% (3660/3660), done.


In [0]:
sms = pd.read_csv('/content/nlpia/src/nlpia/data/sms-spam.csv')

In [4]:
sms.head()

,Unnamed: 0,spam,text
0,0,0,"Go until jurong point, crazy.. Available only ..."
1,1,0,Ok lar... Joking wif u oni...
2,2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,3,0,U dun say so early hor... U c already then say...
4,4,0,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
sms.columns

Index(['Unnamed: 0', 'spam', 'text'], dtype='object')

In [0]:
sms.drop(columns='Unnamed: 0', inplace=True)

In [0]:
index = ['sms{}{}'.format(i, '!'*j) for (i,j) in zip(range(len(sms)), sms.spam)]

In [0]:
sms = pd.DataFrame(sms.values, columns=sms.columns, index=index)
sms['spam'] = sms.spam.astype(int)

In [9]:
print(len(sms))
print(sms.spam.sum())

4837
638


In [10]:
sms.head()

,spam,text
sms0,0,"Go until jurong point, crazy.. Available only ..."
sms1,0,Ok lar... Joking wif u oni...
sms2!,1,Free entry in 2 a wkly comp to win FA Cup fina...
sms3,0,U dun say so early hor... U c already then say...
sms4,0,"Nah I don't think he goes to usf, he lives aro..."


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import casual_tokenize
tfidf_model = TfidfVectorizer(tokenizer=casual_tokenize)
tfidf_docs = tfidf_model.fit_transform(raw_documents=sms.text).toarray()

In [12]:
tfidf_docs.shape

(4837, 9232)

In [13]:
tfidf_docs

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [0]:
mask = sms.spam.astype(bool).values

In [15]:
mask.shape

(4837,)

In [0]:
spam_centroid = tfidf_docs[mask].mean(axis=0) #This is taking word vectors that are across the rows for which mean needs to be calculated.
ham_centroid = tfidf_docs[~mask].mean(axis=0)

In [17]:
spam_centroid.round(2).shape

(9232,)

In [18]:
ham_centroid.round(2).shape

(9232,)

In [19]:
spamminess_score = tfidf_docs.dot(spam_centroid - ham_centroid)

spamminess_score.round(2)

array([-0.01, -0.02,  0.04, ..., -0.01, -0.  ,  0.  ])

In [20]:
spamminess_score.shape

(4837,)

In [0]:
from sklearn.preprocessing import MinMaxScaler

sms['lda_score'] = MinMaxScaler().fit_transform(spamminess_score.reshape(-1,1))

In [0]:
sms['lda_predict'] = (sms.lda_score >0.5).astype(int)

In [23]:
sms.head()

,spam,text,lda_score,lda_predict
sms0,0,"Go until jurong point, crazy.. Available only ...",0.227478,0
sms1,0,Ok lar... Joking wif u oni...,0.177888,0
sms2!,1,Free entry in 2 a wkly comp to win FA Cup fina...,0.718785,1
sms3,0,U dun say so early hor... U c already then say...,0.184565,0
sms4,0,"Nah I don't think he goes to usf, he lives aro...",0.286944,0


In [24]:
1. - (sms.lda_predict - sms.spam).abs().sum()/len(sms)

0.9774653710977879

In [54]:
from pugnlp.stats import Confusion
Confusion(sms['spam lda_predict'.split()])

ModuleNotFoundError: ignored

# LDiA Algorithm on SMS Dataset

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import casual_tokenize
np.random.seed(42)

In [0]:
counter = CountVectorizer(tokenizer=casual_tokenize)
bow_docs = pd.DataFrame(counter.fit_transform(raw_documents=sms.text).toarray(), index=index)

In [27]:
bow_docs.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,9192,9193,9194,9195,9196,9197,9198,9199,9200,9201,9202,9203,9204,9205,9206,9207,9208,9209,9210,9211,9212,9213,9214,9215,9216,9217,9218,9219,9220,9221,9222,9223,9224,9225,9226,9227,9228,9229,9230,9231
sms0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
sms1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
sms2!,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
sms3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
sms4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
counter.vocabulary_.values()

dict_values([3807, 8487, 4675, 6296, 13, 2549, 21, 1531, 5910, 4396, 1973, 5594, 3894, 8977, 4811, 3056, 1971, 25, 2277, 8071, 3855, 1296, 8736, 5874, 4848, 4642, 8875, 8395, 5906, 3604, 3195, 471, 1054, 8933, 2386, 8192, 8890, 3328, 2608, 3450, 8180, 497, 5272, 487, 15, 8020, 948, 6688, 6574, 9, 7651, 8379, 6628, 10, 7889, 7, 2020, 1383, 115, 6003, 438, 8, 6959, 3041, 7034, 7438, 3069, 4207, 2019, 1268, 8065, 5606, 4311, 2948, 8092, 4048, 3819, 8537, 5004, 1435, 4104, 8111, 3613, 4116, 2666, 4535, 1693, 591, 8788, 5784, 1310, 5732, 8967, 1584, 0, 4312, 4954, 7454, 3677, 9158, 8489, 3552, 4533, 7674, 1037, 7955, 9097, 2230, 7127, 9216, 344, 6641, 3240, 5584, 1942, 4519, 5769, 7529, 8918, 5281, 8083, 8312, 1214, 6106, 1452, 6148, 9171, 6796, 5315, 5968, 5386, 5807, 8599, 4022, 7154, 2047, 1253, 2046, 6418, 11, 982, 2489, 3634, 8900, 8569, 5678, 2620, 4036, 7113, 6689, 986, 6450, 6851, 2283, 2038, 263, 2344, 4771, 8565, 384, 4226, 3965, 5441, 371, 5484, 5946, 5489, 6590, 3192, 8495, 8052

In [29]:
counter.vocabulary_.keys()

dict_keys(['go', 'until', 'jurong', 'point', ',', 'crazy', '..', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', '...', 'cine', 'there', 'got', 'amore', 'wat', 'ok', 'lar', 'joking', 'wif', 'u', 'oni', 'free', 'entry', '2', 'a', 'wkly', 'comp', 'to', 'win', 'fa', 'cup', 'final', 'tkts', '21st', 'may', '2005', '.', 'text', '87121', 'receive', 'question', '(', 'std', 'txt', 'rate', ')', 't', '&', "c's", 'apply', '08452810075', 'over', '18', "'", 's', 'dun', 'say', 'so', 'early', 'hor', 'c', 'already', 'then', 'nah', 'i', "don't", 'think', 'he', 'goes', 'usf', 'lives', 'around', 'here', 'though', 'freemsg', 'hey', 'darling', "it's", 'been', '3', "week's", 'now', 'and', 'no', 'word', 'back', '!', "i'd", 'like', 'some', 'fun', 'you', 'up', 'for', 'it', 'still', '?', 'tb', 'xxx', 'chgs', 'send', '£', '1.50', 'rcv', 'even', 'my', 'brother', 'is', 'not', 'speak', 'with', 'me', 'they', 'treat', 'aids', 'patent', 'as', 'per', 'your', 'request', 'melle', 'oru', 'mi

In [0]:
column_nums, terms = zip(*sorted(zip(counter.vocabulary_.values(), counter.vocabulary_.keys())))

In [0]:
bow_docs.columns = terms

In [32]:
bow_docs.head(5)

,!,"""",#,#150,#5000,$,%,&,',(,),*,+,",",-,.,. .,. . .,. . . .,. . . . .,. ..,..,.. .,.. . . .,.. ... ...,...,... . . . .,/,0,00,00870405040,0089,01,0121 2025050,01223585236,01223585334,01256987,02,02/06,02/09,...,yup,yupz,z,zac,zaher's,zealand,zebra,zed,zeros,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada,|,~,,,,,,¡,£,»,èn,é,ú1,ü,ü'll,–,—,‘,’,“,…,┾,〨ud,鈥
sms0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
sms1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
sms2!,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
sms3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
sms4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [33]:
sms.loc['sms0'].text

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [34]:
bow_docs.loc['sms0'][bow_docs.loc['sms0'] > 0].head()


,            1
..           1
...          2
amore        1
available    1
Name: sms0, dtype: int64

# LDiA

In [0]:
from sklearn.decomposition import LatentDirichletAllocation as LDiA

In [0]:
ldia = LDiA(n_components=16, learning_method='batch')

In [0]:
ldia = ldia.fit(bow_docs)

In [38]:
ldia.components_.shape

(16, 9232)

In [0]:
columns = ['topic{}'.format(i) for i in range(ldia.n_components)]

In [0]:
components = pd.DataFrame(ldia.components_.T, index=terms, columns=columns)

In [41]:
components.round(2).head(5)

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic12,topic13,topic14,topic15
!,184.03,15.00,72.22,394.95,45.48,36.14,9.55,44.81,0.43,90.23,37.42,44.18,64.40,297.29,41.16,11.70
"""",0.68,4.22,2.41,0.06,152.35,0.06,0.06,0.06,0.45,0.68,8.42,11.42,0.07,62.72,12.27,0.06
#,0.06,0.06,0.06,0.06,0.06,2.07,0.06,0.06,0.06,0.06,0.06,0.06,1.07,4.05,0.06,0.06
#150,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,1.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06
#5000,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,3.06,0.06,0.06,0.06,0.06,0.06,0.06


In [42]:
components.topic3.sort_values(ascending=False)[:10]

!       394.952246
.       218.049724
to      119.533134
u       118.857546
call    111.948541
£       107.358914
,        96.954384
*        90.314783
your     90.215961
is       75.750037
Name: topic3, dtype: float64

In [0]:
ldia16_topic_vectors = ldia.transform(bow_docs)

In [0]:
ldia16_topic_vectors = pd.DataFrame(ldia16_topic_vectors,index=index, columns=columns)


In [45]:
ldia16_topic_vectors.round(2).head(10)

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic12,topic13,topic14,topic15
sms0,0.00,0.62,0.00,0.00,0.00,0.00,0.00,0.00,0.34,0.00,0.00,0.00,0.00,0.00,0.00,0.00
sms1,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.78,0.01,0.01,0.12,0.01,0.01,0.01,0.01
sms2!,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.98,0.00,0.00,0.00,0.00,0.00,0.00
sms3,0.00,0.00,0.00,0.00,0.09,0.00,0.00,0.00,0.85,0.00,0.00,0.00,0.00,0.00,0.00,0.00
sms4,0.39,0.00,0.33,0.00,0.00,0.00,0.14,0.00,0.00,0.00,0.00,0.00,0.09,0.00,0.00,0.00
sms5!,0.00,0.00,0.28,0.00,0.00,0.00,0.00,0.17,0.00,0.26,0.05,0.00,0.11,0.08,0.05,0.00
sms6,0.00,0.00,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.45,0.00,0.00
sms7,0.00,0.00,0.00,0.00,0.97,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
sms8!,0.57,0.00,0.00,0.16,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00
sms9!,0.00,0.00,0.00,0.43,0.00,0.00,0.00,0.00,0.00,0.11,0.00,0.00,0.00,0.44,0.00,0.00


# LDiA + LDA = spam classifier

In [0]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(ldia16_topic_vectors, sms.spam, test_size=0.5, random_state=271828)

In [92]:
lda = LDA(n_components=1)
lda = lda.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [0]:
sms['ldia16_spam'] = lda.predict(ldia16_topic_vectors)

In [94]:
round(float(lda.score(X_test,y_test)),2)

0.94

In [95]:
round(float(lda.score(X_train,y_train)),2)

0.92

In [0]:
# As there was a warning of multi colinearity. We can find out which two pair of words have exact same tokens just in different order.

from itertools import product
all_pairs = [(word1,word2) for (word1,word2) in product(bow_docs.columns, bow_docs.columns) if not word1 == word2]

In [52]:
len(all_pairs)

85220592

In [0]:
a = [(i,j) for (i,j) in all_pairs if i == j]

In [0]:
import collections
col =  collections.Counter(bow_docs)
a = [item for (key,val) in col.items() if val > 1]

A fairer comparison: 32 LDiA topics

In [74]:
ldia32 = LDiA(n_components=32, learning_method='batch')
ldia32 = ldia32.fit(bow_docs)
ldia32.components_.shape

(32, 9232)

In [77]:
# bow_docs.T.shape
ldia32_topic_vectors.shape

(4837, 32)

In [0]:
ldia32_topic_vectors = ldia32.transform(bow_docs)
columns32 = ['topic{}'.format(i) for i in range(len(ldia32.components_))]

In [0]:
ldia32_topic_vectors = pd.DataFrame(ldia32_topic_vectors, columns = columns32, index=index)

In [87]:
ldia32_topic_vectors.round(2).head()

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic12,topic13,topic14,topic15,topic16,topic17,topic18,topic19,topic20,topic21,topic22,topic23,topic24,topic25,topic26,topic27,topic28,topic29,topic30,topic31
sms0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.4,0.0,0.46
sms1,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.16,0.0,0.0,0.0,0.0,0.0,0.0,0.73
sms2!,0.0,0.98,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
sms3,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.08,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.85
sms4,0.0,0.00,0.0,0.0,0.78,0.0,0.0,0.16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00


In [88]:
X_train,X_test,y_train,y_test = train_test_split(ldia32_topic_vectors, sms.spam, test_size=0.5, random_state=42)
lda = LDA(n_components=1)
lda = lda.fit(X_train,y_train)
sms['lda32_spam'] = lda.predict(ldia32_topic_vectors)

/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [89]:
round(float(lda.score(X_test,y_test)),2)

0.93

In [90]:
round(float(lda.score(X_train,y_train)),2)

0.94